In [16]:
import torch
import torchvision.models as models
import torch.nn as nn
from voc12 import my_dataloader
import os
from utility.image_util import *
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

In [53]:
class DepthResNet50(nn.Module):
    def __init__(self, num_classes=20):
        super(DepthResNet50, self).__init__()

        self.resnet50 = models.resnet50(pretrained=True)
        
        self.resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        self.features = nn.Sequential(*list(self.resnet50.children())[:-1])  # 마지막 FC layer 제외
        
        self.FC = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        x = self.features(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.FC(x)
        
        return x

# 모델 생성
model = DepthResNet50()
criterion = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


In [54]:
img_name_list_path = "voc12/val.txt"
voc12_root = "../Dataset/VOC2012/"
depth_root = "../result/depth_img/"

In [55]:
dataset = my_dataloader.VOC12_DepthClassificationDataset(img_name_list_path, voc12_root, depth_root, crop_size = 224)

In [56]:
data_loader = DataLoader(dataset, batch_size=4)

In [61]:
for step, pack in enumerate(tqdm(data_loader)):
    img = pack['img']
    label = pack['label']
    img = img[:,3].unsqueeze(1)
    
    label = torch.argmax(label, dim = 1)
    print(label.shape)

  0%|          | 0/729 [00:00<?, ?it/s]

  1%|          | 4/729 [00:00<00:38, 19.01it/s]

torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4])


  1%|▏         | 10/729 [00:00<00:34, 21.00it/s]

torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4])


...

  4%|▎         | 26/729 [00:01<00:45, 15.48it/s]

torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4])


  4%|▍         | 30/729 [00:01<00:42, 16.58it/s]

torch.Size([4])
torch.Size([4])
torch.Size([4])


ValueError: zero-dimensional arrays cannot be concatenated